# 简介

本文是B站课程[《PyTorch深度学习实践》完结合集](https://www.bilibili.com/video/BV1Y7411d7Ys/?spm_id_from=333.999.0.0&vd_source=f9eb99d14a0acbcfa188c1e70864412e)的笔记，以及后期自己使用学习的补充，其中包括的内容为：

* torch 的数组构建，以及反向传播的实现
* 标准的全连接网络结构
* 数据集Dataset、DataLoader
* 卷积神经网络实现
* 循环神经网络
* 模型保存与读取

以下是必要引入的库函数

In [ ]:
import torch                        
import numpy as np      #熟悉的数据处理工具
import matplotlib.pyplot as plt     #画图工具
from torch.utils.data import Dataset        #读取数据集 
from torch.utils.data import DataLoader     #将数据集以mini-batch的方式进行训练


# Torch 数组构建
torch 构建的数组，分为两个部分，一个是数据的数组，另一部分是保存梯度的数组。默认梯度数组是不构建的，需要手动选择。
反向传播的过程中需要定义损失函数，而且在之后需要将梯度清零，否则将会与之前的梯度相加。

以下考虑任务：拟合$y = w x$中的$w$

In [ ]:
w = torch.tensor([10.0])            # w为需要优化的参数，首先赋予初值
w.requires_grad = True              # 打开梯度开关

x_data = [1.0, 2.0, 3.0]
y_data = [2.0, 4.0, 6.0]

# 前向传播过程
def forward(x):                     
    return x * w

# 定义损失函数
def loss(x, y):
    y_pred = forward(x)
    return (y_pred - y) ** 2

# tensor的数组，如果只想输出标量值，要用item()，否则将会携带梯度。长期以往，内存占用过高。
print("predict (before training)", 4, forward(4).item()) 

for epoch in range(100):
    for x, y in zip(x_data, y_data):
        l = loss(x, y)
        l.backward()
        print('\tgrad:', x, y, w.data, w.grad.item())
        w.data = w.data - 0.01 * w.grad.data            # 利用梯度数据更新
        w.grad.data.zero_()             # 将梯度置零，否则将会与之前的梯度叠加

注意：从其他地方读入的数组需要转化为 float32 的类型

# 标准的全连接网络结构

应该包含三个部分：
* 网络的构建
* 损失函数
* 优化器（反向传播的过程）

## 网络构建

网络结构应该继承自**torch.nn.Module**，其中必须包含两部分：初始函数 **\_\_init\_\_** 、前向传播 **forward** 。

In [ ]:
class LinearModel(torch.nn.Module):
    def __init__(self):
        super(LinearModel, self).__init__()
        self.linear = torch.nn.Linear(1, 1)

    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred
model = LinearModel()               # 实例化

## 损失函数、优化器

这部分考虑自己的任务是什么（回归、分类），标签的特征是什么，从而选择对应的损失函数与优化器。

In [ ]:
criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

以下考虑任务：拟合$y = w x$中的$w$

In [ ]:
x_data = torch.tensor([[1.0], [2.0]])
y_data = torch.tensor([[3.0], [6.0]])

loss_list = []
for epoch in range(50):
    y_pred = model(x_data)              # 得到预测值
    loss = criterion(y_pred, y_data)    # 计算损失函数

    optimizer.zero_grad()               # 在反向传播前将网络中参数的梯度置0
    loss.backward()                     # 损失进行反向传播
    optimizer.step()                    # 优化器对每一个参数进行更新

# 数据集

Dataset是抽象类用于读取全部的数据集，划分输入数据与目标，无法实例化；DataLoader用以设置mini-batch生成的重要参数，可以实例化。

In [ ]:
class DiabetesDataset(Dataset):
    def __init__(self, filepath):
        xy = np.loadtxt(filepath, delimiter=',', dtype=np.float32)  # 从外部读取
        self.len = xy.shape[0]                          #确定数据的个数
        self.x_data = torch.from_numpy(xy[:, :-1])      #设置其中一组数据
        self.y_data = torch.from_numpy(xy[:, [-1]])

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]   # 返回的数据

    def __len__(self):
        return self.len     # 数据总长度

dataset = DiabetesDataset(PATH)

#实例化，并且设置mini-batch的重要参数
train_loader = DataLoader(dataset=dataset, batch_size=32, shuffle=True, num_workers=4) 

for epoch in range(100):
    for i, data in enumerate(train_loader):     #读取数据
        inputs, labels = data
        y_pred = model(inputs)
        print(y_pred)
        loss = criterion(y_pred, labels)
        #print(epoch, i, loss.item())

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

# 卷积神经网络
标准化的过程包含五个部分：CBAPD，分别是:Conv, BatchNormal, Activate, Pool, Dropout

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size=3)
        self.conv2 = torch.nn.Conv2d(10, 20, kernel_size=2)
        self.conv3 = torch.nn.Conv2d(20, 30, kernel_size=3)
        self.pooling = torch.nn.MaxPool2d(2)
        self.l1 = torch.nn.Linear(120, 80)
        self.l2 = torch.nn.Linear(80, 40)
        self.l3 = torch.nn.Linear(40, 10)
    def forward(self, input):
        batch_size = input.size(0)
        x = F.relu(self.pooling(self.conv1(input)))
        x = F.relu(self.pooling(self.conv2(x)))
        x = F.relu(self.pooling(self.conv3(x)))
        x = x.view(batch_size, -1)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = self.l3(x)
        return x

model = Net()

# 循环神经网络

这部分暂时不熟悉，暂且只展示代码。

## 使用RNNCell单元构建

In [ ]:
cell = torch.nn.RNNCell(input_size=input_size, hidden_size=hidden_size)

## 使用RNN单元构建

In [ ]:
cell = torch.nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)

# 模型保存与读取

详细参考官网的[教程](https://pytorch.org/tutorials/beginner/saving_loading_models.html)

建议保存为：pt、pth后缀

## 只保存模型参数

In [ ]:
# save
torch.save(model.state_dict(), PATH)

# load
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.eval()

## 保存整个模型

有个疑惑：在加载整个模型的时候，是否还需要写出这个模型的类？

In [ ]:
# save
torch.save(model, PATH)

# load
# Model class must be defined somewhere
model = torch.load(PATH)
model.eval()

# 其它

## 如何使用GPU计算

首先确认是否有可用的GPU
然后选择一个设备，在训练的时候，数据集与模型必须在同一个设备上。

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)
data.to(device)